In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import KFold, StratifiedKFold

In [ ]:
df_org = pd.read_csv("train.csv", encoding="cp949")

In [ ]:
df_org.head()

# 범주형 변수 수치화

In [ ]:
df_org = df_org.fillna('없음')

In [ ]:
df_z = df_org['특송업체부호'].value_counts(normalize=True).rename_axis('unique').reset_index(name='counts')
df_un = df_z['unique']
df_co = df_z['counts'].round(decimals=6)
du = df_un.tolist()
dc = df_co.tolist()
for i in range(df_z.shape[0]):
    df_org.loc[df_org['특송업체부호'] == du[i] , '특송업체부호비율'] = dc[i]
df_org['특송업체부호비율'] = df_org['특송업체부호비율'].apply(lambda x: np.log1p(x))

In [ ]:
df_z = df_org['통관지세관부호'].value_counts(normalize=True).rename_axis('unique').reset_index(name='counts')
df_un = df_z['unique']
df_co = df_z['counts'].round(decimals=6)
du = df_un.tolist()
dc = df_co.tolist()
for i in range(df_z.shape[0]):
    df_org.loc[df_org['통관지세관부호'] == du[i] , '통관지세관부호비율'] = dc[i]
df_org['통관지세관부호비율'] = df_org['통관지세관부호비율'].apply(lambda x: np.log1p(x))

In [ ]:
df_z = df_org['해외거래처부호'].value_counts(normalize=True).rename_axis('unique').reset_index(name='counts')
df_un = df_z['unique']
df_co = df_z['counts'].round(decimals=6)
du = df_un.tolist()
dc = df_co.tolist()
for i in range(df_z.shape[0]):
    df_org.loc[df_org['해외거래처부호'] == du[i] , '해외거래처부호비율'] = dc[i]
df_org['해외거래처부호비율'] = df_org['해외거래처부호비율'].apply(lambda x: np.log1p(x))

In [ ]:
df_z = df_org['운송수단유형코드'].value_counts(normalize=True).rename_axis('unique').reset_index(name='counts')
df_un = df_z['unique']
df_co = df_z['counts'].round(decimals=6)
du = df_un.tolist()
dc = df_co.tolist()
for i in range(df_z.shape[0]):
    df_org.loc[df_org['운송수단유형코드'] == du[i] , '운송수단유형코드비율'] = dc[i]
df_org['운송수단유형코드비율'] = df_org['운송수단유형코드비율'].apply(lambda x: np.log1p(x))

In [ ]:
df_org.head(5)

# 변수 Drop

In [ ]:
del df_org['통관지세관부호']
del df_org['특송업체부호']
del df_org['해외거래처부호']
del df_org['운송수단유형코드']
del df_org['신고일자']
del df_org['검사결과코드']
del df_org['핵심적발']
del df_org['신고인부호']
del df_org['신고번호']
del df_org['원산지국가코드']
del df_org['관세율']

In [ ]:
df_org.columns

In [ ]:
# 범주형 변수 지정
discrete_columns = ['수입자부호', '수입통관계획코드', 
       '수입신고구분코드', '수입거래구분코드',
       '수입종류코드', '징수형태코드', 
       '반입보세구역부호', 
       'HS10단위부호', '적출국가코드', '관세율구분코드']

# 범주형 변수들을 문자열로 지정 ('object' -> 'string')
for var in discrete_columns:
    df_org[var] = df_org[var].astype(str)

# 레이블 인코딩

In [ ]:
# 범주형 변수 전처리 1단계: label encoding --> 각 범주형 변수가 갖는 클래스에 고유의 식별번호 부여
# 예) 수입자 상호: AAABB -> 1, 가나다라 -> 2, ...

label_encoding_ref = {}
for var in discrete_columns:
    label_encoding_ref[var] = {code: i+1 for i, code in enumerate(df_org[var].unique())}
    print(label_encoding_ref[var])
    df_org[var] = [label_encoding_ref[var][x] for x in df_org[var]]

In [ ]:
df_org.head()

In [ ]:
# 수치형 변수 지정
numeric_columns = ['신고중량(KG)', '과세가격원화금액', '특송업체부호비율', 
                   '통관지세관부호비율', '해외거래처부호비율', '운송수단유형코드비율']

In [ ]:
# 수치형 변수, 범주형 변수 분리
numeric = df_org[numeric_columns]

In [ ]:
# 수치형 변수 정규화
from sklearn.preprocessing import StandardScaler, RobustScaler
std_scaler = StandardScaler()
rob_scaler = RobustScaler()

numeric['신고중량(KG)'] = rob_scaler.fit_transform(numeric['신고중량(KG)'].values.reshape(-1,1))
numeric['과세가격원화금액'] = rob_scaler.fit_transform(numeric['과세가격원화금액'].values.reshape(-1,1))
numeric['특송업체부호비율'] = rob_scaler.fit_transform(numeric['특송업체부호비율'].values.reshape(-1,1))
numeric['통관지세관부호비율'] = rob_scaler.fit_transform(numeric['통관지세관부호비율'].values.reshape(-1,1))
numeric['해외거래처부호비율'] = rob_scaler.fit_transform(numeric['해외거래처부호비율'].values.reshape(-1,1))
numeric['운송수단유형코드비율'] = rob_scaler.fit_transform(numeric['운송수단유형코드비율'].values.reshape(-1,1))

In [ ]:
df_org.head()

# 수치형 PCA

In [ ]:
from sklearn import decomposition
pca = decomposition.PCA(n_components=5).fit(numeric)

reduced_numeric = pca.transform(numeric)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
reduced_numeric.shape

In [ ]:
df_org['신고중량(KG)']  = reduced_numeric[:, 0]
df_org['과세가격원화금액']  = reduced_numeric[:, 1]
df_org['특송업체부호비율']  = reduced_numeric[:, 2]
df_org['통관지세관부호비율']  = reduced_numeric[:, 2]
df_org['해외거래처부호비율']  = reduced_numeric[:, 3]
df_org['운송수단유형코드비율']  = reduced_numeric[:, 4]

# 데이터셋 확인

In [ ]:
df_org.head(5)

# 훈련/시험 데이터셋 분할

In [ ]:
y = df_org['우범여부']
x = df_org.drop(columns = '우범여부')
type(x), type(y)

In [ ]:
df_org_train, df_org_test, org_train_y, org_test_y = train_test_split(x,y, test_size = 0.3, stratify = y, random_state =3)

# 원핫인코딩 데이터셋 프로세스

In [ ]:
df_org_train['label'] = 'train'
df_org_test['label'] = 'test'

concat_df = pd.concat([df_org_train,df_org_test])

df_org_trains = pd.get_dummies(concat_df, columns = discrete_columns)

df_train =  df_org_trains[df_org_trains['label'] == 'train']
df_test =  df_org_trains[df_org_trains['label'] == 'test']

In [ ]:
df_train = df_train.drop('label', axis=1)
df_test = df_test.drop('label', axis=1)

In [ ]:
# 데이터 사이즈 확인
print(df_train.shape, df_test.shape)
print(org_train_y.shape, org_test_y.shape)

# Imbalanced Sampling
# * SMOTE *

In [ ]:
from imblearn.over_sampling import *

In [ ]:
df_train.info()

In [ ]:
smote = SMOTE(random_state=11, sampling_strategy='all')
df_train_over_x, df_train_over_y = smote.fit_resample(df_train, org_train_y)
df_train_over_x.info(), df_train_over_y

# 모델 생성 / 학습 / 평가

In [ ]:
# XGBoost 모델 생성
from xgboost import XGBClassifier
from sklearn.metrics import f1_score,roc_auc_score
import warnings
warnings.filterwarnings("ignore")

In [ ]:
xgb_clf = XGBClassifier(n_estimators=50, max_depth=4, n_jobs=-1, reg_lambda=1)

eval_set = [(df_train_over_x, df_train_over_y), (df_test, org_test_y)]

xgb_clf.fit(df_train_over_x, df_train_over_y, eval_metric=["logloss"],
            eval_set=eval_set, verbose=True)

In [ ]:
# 모델 성능 시각화

from matplotlib import pyplot
# retrieve performance metrics
results = xgb_clf.evals_result()
epochs = len(results['validation_0']['logloss'])
x_axis = range(0, epochs)
# plot log loss
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
ax.legend()
pyplot.ylabel('Log Loss')
pyplot.title('XGBoost Log Loss')

In [ ]:
# evaluate xgboost model
print("------Evaluating xgboost model------")
# Predict
test_pred = xgb_clf.predict_proba(df_test)[:,1]
# Calculate auc
xgb_auc = roc_auc_score(org_test_y, test_pred)
print(xgb_auc)

In [ ]:
def inspection_performance(predicted_fraud, test_fraud):
        
    Inspect_Rate=[]
    Precision=[]
    Recall=[]
    
    for i in range(0,101,1):
        
        threshold = np.percentile(predicted_fraud, i)
        # Precision = number of frauds / number of inspection
        precision = np.mean(test_fraud[predicted_fraud >= threshold])
        # Recall = number of inspected frauds / number of frauds
        recall = sum(test_fraud[predicted_fraud >= threshold])/sum(test_fraud)
        # Save values
        Inspect_Rate.append(100-i)
        Precision.append(precision)
        Recall.append(recall)
        
    
    compiled_conf_matrix = pd.DataFrame({
        'Inspect_Rate':Inspect_Rate,
        'Precision':Precision,
        'Recall':Recall
    })

    return compiled_conf_matrix

In [ ]:
basic_performance = inspection_performance(test_pred, org_test_y.astype(float))

In [ ]:
# 검사율 1~10% 지정 시 Precision 및 Recall 분석
# Precision (적중률) = (검사선별된 우범건수)/(검사선별 건수)
# Recall (적발률) = (검사선별된 우범건수) / (전체 우범건수)
basic_performance.iloc[range(99,89,-1),:]

In [ ]:
data = pd.melt(basic_performance, 
               id_vars = ['Inspect_Rate'],
               value_vars = ['Recall','Precision'])

sns.relplot(data=data,
            kind='line',
            x="Inspect_Rate", 
            y="value", 
            hue='variable',
            col="variable")

In [ ]:
# 모델에 기여도가 높은 변수 시각화
from xgboost import plot_importance
plt.rcParams["font.family"] = 'Malgun Gothic'
plt.rcParams["figure.figsize"] = (15,10)
plot_importance(xgb_clf, max_num_features=30)
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
y_preds = xgb_clf.predict(df_test)

In [ ]:
print("accuracy score for trained data",accuracy_score(df_train_over_y,xgb_clf.predict(df_train_over_x)))
print("accuracy score is",accuracy_score(org_test_y,y_preds))
print("Confusion matrix",confusion_matrix(org_test_y,y_preds))
print("Report",classification_report(org_test_y,y_preds))

# Submission 값 추출

In [ ]:
print("accuracy score for trained data",accuracy_score(df_train_over_y,xgb_clf.predict(df_train_over_x)))
print("accuracy score is",accuracy_score(org_test_y,y_preds))
print("Confusion matrix",confusion_matrix(org_test_y,y_preds))
print("Report",classification_report(org_test_y,y_preds))

# Submission 값 추출

### Submission 용 Test 데이터셋 추출

In [ ]:
# df_org를 불러올 때 기존 train + test 셋을 합쳐서 불러옴
# 불러온 후 기존에 있던 데이터 전처리 모두 시행

# df_org = pd.read_csv("train+test.csv", encoding="cp949")

In [81]:
# Train + Test의 데이터를 전부 One Hot encoding 시행
df_onehot = pd.get_dummies(df_org, columns = discrete_columns)
df_onehot.tail()

,신고중량(KG),과세가격원화금액,우범여부,특송업체부호비율,통관지세관부호비율,해외거래처부호비율,운송수단유형코드비율,수입자부호_1,수입자부호_2,수입자부호_3,...,관세율구분코드_26,관세율구분코드_27,관세율구분코드_28,관세율구분코드_29,관세율구분코드_30,관세율구분코드_31,관세율구분코드_32,관세율구분코드_33,관세율구분코드_34,관세율구분코드_35
23158,420.8,4.690141e+03,NaN,NaN,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23159,19979.0,3.955338e+06,NaN,NaN,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23160,3793.2,2.673082e+06,NaN,NaN,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23161,5148.8,1.147732e+05,NaN,NaN,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23162,13906.2,5.470683e+07,NaN,NaN,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [209]:
# 인코딩한 데이터프레임을 분리하여 'test' 데이터프레임은 유지
# 분리한 이후 남은 train 셋은 기존에 인코딩 프로세스로 진행

test = df_onehot.iloc[76837:, :]
test.shape

(23163, 11648)

In [210]:
# 분리한 테스트셋의 레이블 삭제

test_x = real_test.drop(columns='우범여부')

In [219]:
# 학습시킨 기존 모델에 'test' 데이터프레임 대상으로 predict 시행

y_preds = xgb_clf.predict(test_x)
y_preds

array([0., 1., 1., ..., 1., 0., 1.])

In [220]:
# 예측결과 데이터프레임 변환 

pred_df = pd.DataFrame(y_preds, columns=['우범여부'])
pred_df

,우범여부
0,0.0
1,1.0
2,1.0
3,0.0
4,1.0
...,...
23158,0.0
23159,1.0
23160,1.0
23161,0.0


In [221]:
# 예측결과 데이터프레임 csv파일 변환

pred_df.to_csv('C:\\Users\\CEO\\result.csv', index = None)